Побудувати логістичну регресію за допомогою нейронної мережі з використанням бібліотеки Pytorch
1. Завантажити датасет «Вилікувати принцесу»
https://www.kaggle.com/datasets/unmoved/cure-the-princess?select=data.csv
2. Провести аналіз і підготовку даних.
3. Поділити датасет на навчальну і тестову вибірки, масштабувати ознаки.
4. Сформувати тензори.
5. Побудувати нейронну мережу.
6. Навчити.
7. Вивести точність і функцію втрат.

8. Побудувати логістичну регресію за допомогою методів МН.
9. Оцінити метрики.

10. Вивести клас. Звіт, матрицю плутанини.
11. Порівняти результати.
12. Написати Висновки.

In [1]:
!pip install torch torchvision scikit-learn pandas matplotlib seaborn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.1 MB/s eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
# Завантажити датасет «Вилікувати принцесу»
uploaded = files.upload()

Saving data.csv to data (1).csv


In [19]:
df = pd.read_csv("data.csv")
df

,Phoenix Feather,Unicorn Horn,Dragon's Blood,Mermaid Tears,Fairy Dust,Goblin Toes,Witch's Brew,Griffin Claw,Troll Hair,Kraken Ink,Minotaur Horn,Basilisk Scale,Chimera Fang,Cured
0,2.4,18.7,18.4,27.9,7.9,9.6,18.3,13.2,2.5,26.0,10.5,26.2,12.5,0
1,2.1,6.0,15.0,13.3,15.6,13.1,11.0,5.0,7.2,26.0,1.5,13.3,6.2,0
2,17.2,13.9,23.8,6.8,10.7,15.8,19.4,2.7,15.4,21.2,11.1,16.6,11.4,1
3,8.4,9.7,6.8,26.9,4.6,29.1,14.6,19.7,18.0,20.8,13.6,13.9,8.1,1
4,22.1,10.8,16.4,10.5,22.0,23.4,2.6,18.2,23.8,11.3,5.5,16.8,16.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,9.4,2.2,15.8,5.9,29.7,18.7,11.5,13.1,15.3,22.5,10.1,4.7,13.8,0
2334,12.1,7.6,20.6,5.3,18.9,19.1,9.4,11.9,21.8,12.0,26.7,8.4,24.4,1
2335,15.2,33.2,7.2,14.5,16.0,16.7,1.2,32.5,34.5,25.9,3.9,18.0,19.2,1
2336,2.0,17.0,33.2,13.2,29.1,35.5,19.7,30.3,30.7,4.3,15.7,20.5,2.1,1


In [20]:
#Провести аналіз і підготовку даних
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338 entries, 0 to 2337
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Phoenix Feather  2338 non-null   float64
 1   Unicorn Horn     2338 non-null   float64
 2   Dragon's Blood   2338 non-null   float64
 3   Mermaid Tears    2338 non-null   float64
 4   Fairy Dust       2338 non-null   float64
 5   Goblin Toes      2338 non-null   float64
 6   Witch's Brew     2338 non-null   float64
 7   Griffin Claw     2338 non-null   float64
 8   Troll Hair       2338 non-null   float64
 9   Kraken Ink       2338 non-null   float64
 10  Minotaur Horn    2338 non-null   float64
 11  Basilisk Scale   2338 non-null   float64
 12  Chimera Fang     2338 non-null   float64
 13  Cured            2338 non-null   int64  
dtypes: float64(13), int64(1)
memory usage: 255.8 KB


In [21]:
df.isnull().sum()

,0
Phoenix Feather,0
Unicorn Horn,0
Dragon's Blood,0
Mermaid Tears,0
Fairy Dust,0
Goblin Toes,0
Witch's Brew,0
Griffin Claw,0
Troll Hair,0
Kraken Ink,0


In [23]:
X = df.drop('Cured', axis=1).values
y = df['Cured'].values

In [24]:
# Масштабування
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [25]:
# Поділ на навчальну і тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [26]:
# Перетворення у тензори
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [27]:
# Побудова моделі
class LogisticRegressionNN(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionNN, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegressionNN(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [28]:
# Навчання
epochs = 100
for epoch in range(epochs):
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 0.1748
Epoch [20/100], Loss: 0.0949
Epoch [30/100], Loss: 0.2037
Epoch [40/100], Loss: 0.2654
Epoch [50/100], Loss: 0.1555
Epoch [60/100], Loss: 0.5133
Epoch [70/100], Loss: 0.1290
Epoch [80/100], Loss: 0.3383
Epoch [90/100], Loss: 0.3335
Epoch [100/100], Loss: 0.3016


In [29]:
# Тестування PyTorch-моделі
with torch.no_grad():
    y_pred_nn = model(X_test_tensor)
    y_pred_nn_cls = (y_pred_nn >= 0.5).float()

acc_nn = accuracy_score(y_test, y_pred_nn_cls)
print(f"\nТочність PyTorch моделі: {acc_nn:.4f}")


Точність PyTorch моделі: 0.8974


In [30]:
# Логістична регресія з sklearn
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_sk = log_reg.predict(X_test)

acc_sk = accuracy_score(y_test, y_pred_sk)
print(f"\nТочність LogisticRegression з sklearn: {acc_sk:.4f}")


Точність LogisticRegression з sklearn: 0.8932


In [31]:
# Порівняння результатів
print("\nЗвіт PyTorch:")
print(classification_report(y_test, y_pred_nn_cls))

print("\nЗвіт sklearn:")
print(classification_report(y_test, y_pred_sk))


Звіт PyTorch:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90       231
           1       0.90      0.90      0.90       237

    accuracy                           0.90       468
   macro avg       0.90      0.90      0.90       468
weighted avg       0.90      0.90      0.90       468


Звіт sklearn:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       231
           1       0.90      0.89      0.89       237

    accuracy                           0.89       468
   macro avg       0.89      0.89      0.89       468
weighted avg       0.89      0.89      0.89       468



Висновок.
У ході виконання лабораторної роботи було проведено попередній аналіз та підготовку даних датасету «Вилікувати принцесу». Після цього було реалізовано логістичну регресію двома способами: за допомогою нейронної мережі на PyTorch та класичної моделі з використанням sklearn. Обидві моделі показали майже однакові результати, однак результат моделі на PyTorch був трохи вищим — 0.8974 проти 0.8932 у sklearn.